## Exercises

1.    Create a new file named model_evaluation.py or model_evaluation.ipynb for these exercises.

2.    Given the following confusion matrix, evaluate (by hand) the model's performance.

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |

    In the context of this problem, what is a false positive?
    In the context of this problem, what is a false negative?
    How would you describe this model?


In [4]:
# 2. I called Dog positive and cat negative
# - A False Positve is when i predict a dog, but it was actually a cat
# - A False Negative is when I predict a cat, but it was actually a dog.
# - How would you describe this model:

# - Accuracy = .8
# - Recall ~ .87
# - Precision ~ .78

# This model is pretty good! It has better recall than precision though, so if the cost of a false negative
# is something we are worried about, we are in good shape. If we are more worried about a false positive, we still
# have a good model, but not as good.

3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

Use the predictions dataset and pandas to help answer the following questions:

    - Below I calculate the accuracy, recall, and precision for each model
    - questions are pasted and answered further below

In [5]:
import numpy as np
import pandas as pd
import os

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

In [7]:
filename = 'c3.csv'
df = pd.read_csv(filename)
df.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [93]:
# calculate baseline accuracy
   ### next time, just do a value_counts of the actual column. Then pick the one with the highest count 
   ### as your baseline
df['all_pos'] = 'Defect'
df['all_neg'] = 'No Defect'
all_pos_accuracy = (df.actual == df.all_pos).mean()
all_neg_accuracy = (df.actual == df.all_neg).mean()
print ("Baseline accuracy = the greater of predicting all positives or predicting all negatives.")
print (f"The accuracy of predicting all positives is {round(all_pos_accuracy, 2)}")
print (f"The accuracy of predicting all negatives is {round(all_neg_accuracy, 2)}")
baseline_accuracy = max(all_pos_accuracy, all_neg_accuracy)
print (f"Therefore the baseline accuracy is {round(baseline_accuracy, 2)}")

Baseline accuracy = the greater of predicting all positives or predicting all negatives.
The accuracy of predicting all positives is 0.08
The accuracy of predicting all negatives is 0.92
Therefore the baseline accuracy is 0.92


In [94]:
#calculate baseline recall using all_neg column
subset = df[df.actual == 'Defect']
baseline_recall = (subset.actual == subset.all_neg).mean()

#calculate baseline precision
subset = df[df.all_neg == 'Defect']
# baseline_precision = (subset.actual == subset.all_neg).mean() ## Errors out ##

# That errors out. Since we only look at the subset where we made positive prediction for our baseline (all_neg)
# prediction, we get 0. Why? The baseline prediction is to predict all negatives.
baseline_precision = 0.0

In [95]:
#make a metrics_df to collect our metrics. Fill it with the baseline metrics first.

# add baseline accuracy
metrics_df = pd.DataFrame(data = [{'metric' : 'accuracy', 'baseline' : baseline_accuracy}])

# add the baseline recall
new_df = pd.DataFrame(data = [{'metric' : 'recall', 'baseline' : baseline_recall}])
metrics_df = pd.concat([metrics_df, new_df], axis=0, ignore_index=True)

# add the baseline precision
new_df = pd.DataFrame(data = [{'metric' : 'precision', 'baseline' : baseline_precision}])
metrics_df = pd.concat([metrics_df, new_df], axis=0, ignore_index=True)

In [96]:
# calculate accuracy, recall, and precision for each model and append statistics to metrics_df

model_cols = np.array(df.columns)[1:4]
for col in model_cols:
    accuracy = (df[col] == df.actual).mean()
    
    subset = df[df.actual == 'Defect']
    recall = (subset.actual == subset[col]).mean()
    
    subset = df[df[col] == 'Defect']
    precision = (subset.actual == subset[col]).mean()
    
    new_df = pd.DataFrame({col: [accuracy, recall, precision]})
    metrics_df = pd.concat([metrics_df, new_df], axis=1)

In [97]:
metrics_df

,metric,baseline,model1,model2,model3
0,accuracy,0.92,0.95,0.5600,0.555000
1,recall,0.00,0.50,0.5625,0.812500
2,precision,0.00,0.80,0.1000,0.131313


    - An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?
    
### Answer: Here we want to maximize recall to minimize our false negatives. In other words, the cost of a FN is high.  Model3 has the highest rate for recall. Pick Model3

    - Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?
    
### Answer: The cost of a false positive is high, so we want to minimize it. Precision is the metric to use. Model1 has the highest value for precision. Pick Model1

## Question 4

4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

Several models have already been developed with the data, and you can find their results here.

Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

#### Questions pasted below the classification_report printout

In [100]:
filename = 'gives_you_paws.csv'
df = pd.read_csv(filename)
df.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [108]:
df['baseline'] = 'dog'
(df.actual == df.baseline).mean()

0.6508

In [111]:
# using classification_report to try and go fast
model_cols = np.array(df.columns)[1:5]
for col in model_cols:
    print(col)
    print(classification_report(df.actual, df[col], labels = ['cat', 'dog']))

model1
              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84      3254

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000

model2
              precision    recall  f1-score   support

         cat       0.48      0.89      0.63      1746
         dog       0.89      0.49      0.63      3254

    accuracy                           0.63      5000
   macro avg       0.69      0.69      0.63      5000
weighted avg       0.75      0.63      0.63      5000

model3
              precision    recall  f1-score   support

         cat       0.36      0.51      0.42      1746
         dog       0.66      0.51      0.57      3254

    accuracy                           0.51      5000
   macro avg       0.51      0.51      0.50      5000
weighted avg       0.55      0.51      0.52      5000

    In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?
#### Answer: Model 1 and 4 have better than baseline accuracy
    Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?
#### Answer: Model 4. I want to minimize false negatives, so I pick the highest recall value with dog on the left.
    Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?
#### Answer: Model2. I want to minimize false negatives, so I pick the highest recall value with cat on the left.

5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

    sklearn.metrics.accuracy_score
    sklearn.metrics.precision_score
    sklearn.metrics.recall_score
    sklearn.metrics.classification_report